In [9]:
import pandas as pd
import re
from tqdm import tqdm
import os
from konlpy.tag import Mecab
from tensorflow.keras.preprocessing import sequence
from transformers import BertTokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from transformers import TFElectraModel
from sklearn.metrics import classification_report
import numpy as np

import pandas as pd
from tqdm import tqdm

import argparse
import time
import gc
import numpy as np
import collections
import sys
import random
import openpyxl

In [10]:
class TFElectraClassifier(tf.keras.Model):
    def __init__(self, num_class):
        super(TFElectraClassifier, self).__init__()

        self.model = TFElectraModel.from_pretrained('../nscc_mecab_base_final/torch_model', from_pt=True)

        #self.electra_layers = model.get_layer('electra')
        
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.classifier = tf.keras.layers.Dense(num_class, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02), 
                                                name="classifier", activation='softmax')
        
    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        
        #outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.model(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = keras.layers.Lambda(lambda seq: seq[:, 0, :])(outputs[0])
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

In [11]:
desc = 'keyword_classification_22.3.9-1'
lr = 5e-05

In [12]:
main_keywords = pd.read_csv('main_keywords_220210.csv')

In [13]:
main_keywords

,keyword,code,token_keyword,encode_keyword,target,preds,correct,top1,IDF,CFIDF
0,강관 접합부 용접 볼트 편측,EC01,강 ##관 접합부 용접 볼 ##트 편 ##측,"[311, 3700, 13728, 8537, 1671, 3930, 3305, 3701]",['EC01'],"['EC01', 'EB03', 'EA03', 'EA04', 'EA05']",['EC01'],EC01,12.108008,69.993287
1,크론병궤양성 대장염clusterin,LC02,크 ##론 ##병 궤 ##양성 대장 ##염 cluster ##in,"[3085, 3952, 3680, 4442, 10837, 10060, 3862, 1...",['LC02'],"['LC02', 'LC04', 'LA04', 'LC03', 'LC01']",['LC02'],LC02,12.108008,69.993287
2,관심 객체 및 이벤트 검출 및 인식,EE01,관심 객체 및 이벤트 검출 및 인식,"[6513, 8134, 1576, 10031, 6552, 1576, 5907]",['EE01'],"['EE01', 'EE02', 'ED08', 'EA05', 'ED11']",['EE01'],EE01,12.108008,69.993287
3,식물의 성장과 면역 사이의 상호소통,LA01,식물 의 성장 과 면역 사이 의 상호 ##소통,"[6857, 3790, 6304, 3961, 7366, 5919, 3790, 614...",['LA01'],"['LA01', 'LA02', 'LB02', 'LB18', 'LB01']",['LA01'],LA01,12.108008,69.993287
4,미각물질 간 상호작용식품 미각 물질,LB17,미 ##각 ##물질 간 상호 작용 식품 미 ##각 물질,"[1561, 3722, 6367, 298, 6144, 7630, 9005, 1561...",['LB17'],"['LB17', 'LB18', 'LC11', 'LA09', 'LB19']",['LB17'],LB17,12.108008,69.993287
...,...,...,...,...,...,...,...,...,...,...
36548,앤렙,LA02,앤 렙,"[2216, 4337]",['LA02'],"['LA02', 'LB17', 'LB01', 'LA07', 'LA01']",['LA02'],LA02,12.108008,69.993287
36549,봉침,LC07,봉 ##침,"[1675, 3716]",['LC07'],"['LC07', 'EB05', 'LA09', 'LB05', 'LC03']",['LC07'],LC07,11.009395,63.642491
36550,탤린,LA01,탤 ##린,"[3125, 3909]",['LA01'],"['LA01', 'EA15', 'LA02', 'EE14', 'EC04']",['LA01'],LA01,12.108008,69.993287
36551,톡톡,EE02,톡 ##톡,"[3174, 4429]",['EE02'],"['EE02', 'EE09', 'EI05', 'EI11', 'LC08']",['EE02'],EE02,11.009395,63.642491


In [14]:
keywords = main_keywords['keyword']
codes = main_keywords['code']

In [15]:
regex = re.compile('[^ 가-힣a-zA-Z!@#$%^&*,]|[가나다라마바사]\)|[0-9]-?[0-9]?\)')
mecab = Mecab()


f = open('../model/labels.txt', 'r')
labels = []
while True:
    line = f.readline().replace('\n', '')
    if not line:
        break
    labels.append(line)
f.close()

In [16]:
categories = {}
for i, l in enumerate(labels):
    categories[l] = i
print("Train Labels : {}".format(len(categories)))

sample_count = {}
x_data = []
y_data = []
for n, (l, d) in enumerate(zip(codes, keywords)):
    if l[:4] not in categories:
        continue
    else:
        tmp = ''
        for s in d:
            if s == '\n':
                tmp = tmp + " [SEP] "
            else:
                tmp = tmp + s
        x_data.append(tmp)
        y_data.append(categories[l[:4]])
        
str_train_x, str_test_x, train_y, test_y = train_test_split(x_data,y_data, test_size=0.3, random_state=72)

print(f"Train : {len(str_train_x)} / Test : {len(str_test_x)}")

# print(f"Train : {len(x_data)} / Test : {len(y_data)}")

Train Labels : 145
Train : 25587 / Test : 10966


In [17]:
x_data
# y_data
# len(x_data)

['강관  접합부  용접  볼트  편측',
 '크론병궤양성 대장염clusterin',
 '관심 객체 및 이벤트 검출 및 인식',
 '식물의 성장과 면역 사이의 상호소통',
 '미각물질 간 상호작용식품 미각 물질',
 '강 진동 다중선형 특이 적분 연산자',
 'Sonic Hedgehog 신호체계',
 '플렉서블 100G PON 광트랜시버',
 '사람의 표피를 재구성한 인공피부모델',
 '조각구조기반 가상 화합물 라이브러리',
 '다중 자극 유도 하이드로젤 스케폴드',
 '피채널 산화물 반도체 박막트랜지스터',
 '갑상선암음성검사음성 치료삶의질초음파',
 '치과재료 의치상용레진 의료기기 개정',
 'O2O 오프라인 to 온라인 서비스',
 '하이브리드 파워트레인친환경 상용차량',
 '속도 거리 연산 장치열차자동방호장치',
 '지지체 다공성 금속 수소 정제 분리',
 '태양열 해수담수 다중효용 윅 소용량',
 '극초단 광양자빔 응용 분광센서 기술',
 'current commutation',
 '유방암식품 및 식물 내 생리활성물질',
 '환경  골프클럽  농약  토양  물',
 '엘티이 어드밴스 SON Server',
 '엑시톤 홀 효과와 마요라나 페르미온',
 '요소수요소수 펌프선택적 촉매환원장치',
 '융합스테이션 수소 전기 신재생 실증',
 '다중 셀 계층 고밀도 HetSNet',
 '친환경  세정  품질  세제  기술',
 '평형 및 비평형 통계물리학과 열역학',
 '가압식 복강내 에어로졸 항암화학요법',
 '질화갈륨 화학기계연마수소화물기상성장',
 '운영 및 유지보수지식기반 프레임워크',
 '화학  자원  에너지  추출  전기',
 '인체 텔로머레이즈 역전사효소 전사체',
 '실리콘온절연체 기반 실리콘 태양전지',
 '표준모형 너머의 새로운 입자물리이론',
 '유전자 맞춤 바스켓 디자인 임상연구',
 '활동성 은하핵의 플라즈마 물리 연구',
 'HTML5 스마트 TV 플랫폼 표준',
 'GCSF 동원 말초혈액 자가줄기세포',
 '안지오텐신 수용체 네프릴리

In [18]:
categories

{'EA01': 0,
 'EA02': 1,
 'EA03': 2,
 'EA04': 3,
 'EA05': 4,
 'EA06': 5,
 'EA07': 6,
 'EA08': 7,
 'EA09': 8,
 'EA10': 9,
 'EA11': 10,
 'EA13': 11,
 'EA14': 12,
 'EA15': 13,
 'EB01': 14,
 'EB02': 15,
 'EB03': 16,
 'EB04': 17,
 'EB05': 18,
 'EB06': 19,
 'EB07': 20,
 'EC01': 21,
 'EC02': 22,
 'EC03': 23,
 'EC04': 24,
 'EC05': 25,
 'EC06': 26,
 'EC07': 27,
 'EC09': 28,
 'ED01': 29,
 'ED02': 30,
 'ED03': 31,
 'ED04': 32,
 'ED05': 33,
 'ED06': 34,
 'ED07': 35,
 'ED08': 36,
 'ED09': 37,
 'ED10': 38,
 'ED11': 39,
 'EE01': 40,
 'EE02': 41,
 'EE03': 42,
 'EE04': 43,
 'EE05': 44,
 'EE06': 45,
 'EE07': 46,
 'EE08': 47,
 'EE09': 48,
 'EE10': 49,
 'EE11': 50,
 'EE12': 51,
 'EE13': 52,
 'EE14': 53,
 'EF01': 54,
 'EF02': 55,
 'EF04': 56,
 'EF05': 57,
 'EF06': 58,
 'EG04': 59,
 'EG06': 60,
 'EG07': 61,
 'EH01': 62,
 'EH02': 63,
 'EH03': 64,
 'EH06': 65,
 'EH07': 66,
 'EH09': 67,
 'EH10': 68,
 'EH11': 69,
 'EI01': 70,
 'EI02': 71,
 'EI03': 72,
 'EI04': 73,
 'EI05': 74,
 'EI06': 75,
 'EI08': 76,
 'EI09': 

In [19]:
idx_2_cate = {}
for i in categories:
    idx_2_cate[categories[i]] = i
    
analysis_filename = "analysis_test_{desc}.csv".format(desc=desc)
analy_x = []
analy_y = []
# for x, y in zip(str_test_x, test_y):
#     analy_x.append(x)
#     analy_y.append(idx_2_cate[y])

for x, y in zip(keywords, codes):
    analy_x.append(x)
    analy_y.append(y)

df_analy = pd.DataFrame({"input" : analy_x, "target" : analy_y})
df_analy.to_csv(analysis_filename, index=False)

In [20]:
idx_2_cate

{0: 'EA01',
 1: 'EA02',
 2: 'EA03',
 3: 'EA04',
 4: 'EA05',
 5: 'EA06',
 6: 'EA07',
 7: 'EA08',
 8: 'EA09',
 9: 'EA10',
 10: 'EA11',
 11: 'EA13',
 12: 'EA14',
 13: 'EA15',
 14: 'EB01',
 15: 'EB02',
 16: 'EB03',
 17: 'EB04',
 18: 'EB05',
 19: 'EB06',
 20: 'EB07',
 21: 'EC01',
 22: 'EC02',
 23: 'EC03',
 24: 'EC04',
 25: 'EC05',
 26: 'EC06',
 27: 'EC07',
 28: 'EC09',
 29: 'ED01',
 30: 'ED02',
 31: 'ED03',
 32: 'ED04',
 33: 'ED05',
 34: 'ED06',
 35: 'ED07',
 36: 'ED08',
 37: 'ED09',
 38: 'ED10',
 39: 'ED11',
 40: 'EE01',
 41: 'EE02',
 42: 'EE03',
 43: 'EE04',
 44: 'EE05',
 45: 'EE06',
 46: 'EE07',
 47: 'EE08',
 48: 'EE09',
 49: 'EE10',
 50: 'EE11',
 51: 'EE12',
 52: 'EE13',
 53: 'EE14',
 54: 'EF01',
 55: 'EF02',
 56: 'EF04',
 57: 'EF05',
 58: 'EF06',
 59: 'EG04',
 60: 'EG06',
 61: 'EG07',
 62: 'EH01',
 63: 'EH02',
 64: 'EH03',
 65: 'EH06',
 66: 'EH07',
 67: 'EH09',
 68: 'EH10',
 69: 'EH11',
 70: 'EI01',
 71: 'EI02',
 72: 'EI03',
 73: 'EI04',
 74: 'EI05',
 75: 'EI06',
 76: 'EI08',
 77: 'EI0

In [21]:
df_analy

,input,target
0,강관 접합부 용접 볼트 편측,EC01
1,크론병궤양성 대장염clusterin,LC02
2,관심 객체 및 이벤트 검출 및 인식,EE01
3,식물의 성장과 면역 사이의 상호소통,LA01
4,미각물질 간 상호작용식품 미각 물질,LB17
...,...,...
36548,앤렙,LA02
36549,봉침,LC07
36550,탤린,LA01
36551,톡톡,EE02


In [22]:
str_train_x, str_test_x, train_y, test_y = train_test_split(x_data,y_data, test_size=0.3, random_state=72)

In [23]:
len(max(str_train_x, key=len))

19

In [24]:
print('Loading BERT tokenizer...')
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)
# tokenizer = BertTokenizer.from_pretrained('models/paper_kisti_base/vocab.txt', do_lower_case=False)
tokenizer = BertTokenizer.from_pretrained('../nscc_mecab_base_final/vocab.txt', do_lower_case=False) # 최신 vocab


# MAX_LENGTH = len(max(str_train_x, key=len))
MAX_LENGTH = 25
BATCH_SIZE = 32

Loading BERT tokenizer...


/home/ubuntu/anaconda3/envs/pytorch1.4_p36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1646: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,


In [25]:
encode_train_data = []
input_ids =[]
attention_masks =[]
token_type_ids =[]
for line in tqdm(str_train_x):
    line = ' '.join(mecab.morphs(line)) # mecab 적용
    encoded_dict = tokenizer.encode_plus(line, \
                                         add_special_tokens = True,\
                                         pad_to_max_length=True,\
                                         max_length=MAX_LENGTH, 
                                        return_attention_mask=True,
                                        truncation = True)

    input_id=encoded_dict['input_ids']
    attention_mask=encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']
    
    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)

train_input_ids=np.array(input_ids, dtype=int)
train_attention_masks=np.array(attention_masks, dtype=int)
train_token_type_ids=np.array(token_type_ids, dtype=int)
train_x=(train_input_ids, train_attention_masks, train_token_type_ids)

print("Original Text : ", x_data[0])
print("Tokenizer Text : ", tokenizer.tokenize(x_data[0], add_special_tokens = True, max_length=MAX_LENGTH))
print("Encode Text : ", tokenizer.encode(x_data[0], add_special_tokens = True, max_length=MAX_LENGTH))

  0%|          | 0/25587 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/pytorch1.4_p36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 25587/25587 [00:05<00:00, 4914.14it/s]
Keyword arguments {'add_special_tokens': True, 'max_length': 25} not recognized.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) wit

Original Text :  강관  접합부  용접  볼트  편측
Tokenizer Text :  ['강', '##관', '접합부', '용접', '볼', '##트', '편', '##측']
Encode Text :  [202, 311, 3700, 13728, 8537, 1671, 3930, 3305, 3701, 203]


In [26]:
category = set(train_y)
category_idx = dict()
for n, cate in enumerate(category):
    category_idx[cate] = n
idx_train_label = []
for l in tqdm(train_y):
    idx_train_label.append(category_idx[l])
    
train_y = to_categorical(idx_train_label)

print('Train : {} / {}'.format(train_x[0].shape, train_y.shape))

100%|██████████| 25587/25587 [00:00<00:00, 883289.35it/s]

Train : (25587, 25) / (25587, 145)


In [27]:
with tf.device('/device:GPU:1'):
    model = TFElectraClassifier(num_class=len(category))

    # 학습 준비하기
    model.compile(
    #     optimizer=keras.optimizers.Adam(learning_rate=5e-5),
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss = keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy',tf.keras.metrics.TopKCategoricalAccuracy(k=3)]
    )

    checkpoint_filepath = "electra_ckpt_{desc}/ckpt".format(desc=desc)
    checkpoint_dir = os.path.dirname(checkpoint_filepath)
    model.save_weights(checkpoint_filepath.format(epoch=0))

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_loss',
        mode='min',
        save_best_only=True)

    model.fit(train_x, train_y, batch_size=BATCH_SIZE, validation_split=0.2, epochs=100, callbacks=[callback, model_checkpoint_callback])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'electra.embeddings.position_ids', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing TFElectraModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFElectraModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further train

Epoch 1/100
640/640 [==============================] - 61s 95ms/step - loss: 2.9889 - accuracy: 0.3156 - top_k_categorical_accuracy: 0.5094 - val_loss: 2.1181 - val_accuracy: 0.4787 - val_top_k_categorical_accuracy: 0.6866
Epoch 2/100
640/640 [==============================] - 59s 92ms/step - loss: 1.8250 - accuracy: 0.5320 - top_k_categorical_accuracy: 0.7484 - val_loss: 1.8861 - val_accuracy: 0.5496 - val_top_k_categorical_accuracy: 0.7429
Epoch 3/100
640/640 [==============================] - 57s 89ms/step - loss: 1.2224 - accuracy: 0.6677 - top_k_categorical_accuracy: 0.8570 - val_loss: 1.8984 - val_accuracy: 0.5559 - val_top_k_categorical_accuracy: 0.7610
Epoch 4/100
640/640 [==============================] - 57s 89ms/step - loss: 0.7662 - accuracy: 0.7838 - top_k_categorical_accuracy: 0.9324 - val_loss: 1.9911 - val_accuracy: 0.5676 - val_top_k_categorical_accuracy: 0.7624
Epoch 5/100
640/640 [==============================] - 57s 90ms/step - loss: 0.4575 - accuracy: 0.8661 - top

In [28]:
encode_test_data = []
input_ids =[]
attention_masks =[]
token_type_ids =[]
for line in tqdm(str_test_x):
    line = ' '.join(mecab.morphs(line)) # mecab 적용
    encoded_dict = tokenizer.encode_plus(line, \
                                         add_special_tokens = True,\
                                         pad_to_max_length=True,\
                                         max_length=MAX_LENGTH, 
                                        return_attention_mask=True,
                                        truncation = True)

    input_id=encoded_dict['input_ids']
    attention_mask=encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']
    
    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)

test_input_ids=np.array(input_ids, dtype=int)
test_attention_masks=np.array(attention_masks, dtype=int)
test_token_type_ids=np.array(token_type_ids, dtype=int)
test_x=(test_input_ids, test_attention_masks, test_token_type_ids)

  0%|          | 0/10966 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/pytorch1.4_p36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 10966/10966 [00:02<00:00, 4668.49it/s]


In [29]:
idx_test_label = []
for l in tqdm(test_y):
    idx_test_label.append(category_idx[l])
    

test_y = to_categorical(idx_test_label)

print('test : {} / {}'.format(test_x[0].shape, test_y.shape))

100%|██████████| 10966/10966 [00:00<00:00, 861631.25it/s]

test : (10966, 25) / (10966, 145)


In [30]:
with tf.device('/device:GPU:1'):
#     score_his = model.evaluate(test_x, test_y)
    y_pred = model.predict(test_x)

In [31]:
y_pred

array([[2.47295958e-07, 2.86929378e-07, 2.06543334e-07, ...,
        6.94621065e-07, 5.41159295e-07, 6.05293280e-07],
       [8.98609505e-05, 9.23491432e-04, 6.09701929e-05, ...,
        2.98751565e-05, 6.29942951e-06, 8.60604723e-06],
       [1.55079470e-05, 2.60535107e-05, 1.48110792e-06, ...,
        1.56286242e-05, 3.11061240e-06, 1.22975780e-06],
       ...,
       [1.09627979e-06, 7.67859774e-07, 7.58822443e-06, ...,
        1.05328493e-08, 7.29625782e-08, 2.54975348e-08],
       [9.05009165e-06, 1.32024970e-05, 1.17107284e-05, ...,
        1.02677513e-05, 8.72509463e-06, 2.04315961e-06],
       [4.12852094e-02, 4.56294038e-05, 1.34177835e-05, ...,
        3.21279695e-05, 2.65665381e-06, 2.53871781e-06]], dtype=float32)

In [32]:
pred_y = []
for y in y_pred:
    tmp = np.zeros(len(categories))
    tmp[y.argmax()] = 1
    pred_y.append(tmp)

In [33]:
pred_y

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.

In [34]:
y_preds = np.argsort(y_pred, axis=1)[:,-5:]

In [35]:
y_preds

array([[ 83, 109,  85, 107,  80],
       [ 43,  53,  47,  41,  49],
       [ 47,  48,  51,  70,  74],
       ...,
       [ 56,  37,  57,   8,  31],
       [ 81,  80, 109, 108, 107],
       [ 86,  36,  35,   0, 110]])

In [36]:
str_test_y = []
for y in tqdm(test_y):
    str_test_y.append(idx_2_cate[y.argmax()])

100%|██████████| 10966/10966 [00:00<00:00, 442981.20it/s]


In [37]:
result_top5_file_name = 'result_NSCC_top-5_{desc}.csv'.format(desc=desc)
list_preds = []
for i in range(len(y_preds)):
    inner_preds = []
    for j in range(4, -1, -1):
        inner_preds.append(idx_2_cate[y_preds[i][j]])
    list_preds.append(inner_preds)
df_result = pd.DataFrame({'inputs' : str_test_x, 'target' : str_test_y, 'preds' : list_preds})
df_result.to_csv(result_top5_file_name, index=False)

In [38]:
df_result

,inputs,target,preds
0,인슐린신호전달단백질,LA01,"[LA01, LC01, LA06, LC03, LA04]"
1,몰입형 스마트워크,EE04,"[EE10, EE02, EE08, EE14, EE04]"
2,지역단위 교통제어,EI05,"[EI05, EI01, EE12, EE09, EE08]"
3,포도당 운반 단백 단위,LC01,"[LC01, LA07, LC02, LA01, LA02]"
4,사이토카인 아이엘32,LA04,"[LA04, LC01, LC03, LA01, LB07]"
...,...,...,...
10961,방사선 반응 추적,EG07,"[EG07, NB01, EA01, LC04, EG04]"
10962,상향식 감축모형,EH10,"[EF01, EF06, EH02, EI09, EI12]"
10963,멀티레벨 DCDC 컨버터,ED03,"[ED03, EA09, EF05, ED09, EF04]"
10964,항가시아메바성 약제,LC01,"[LC01, LC02, LC03, LA01, LA02]"


In [39]:
top5s = []
top1s = []
for i in tqdm(range(len(df_result['target']))):
    code = df_result['target'][i]
    if code in df_result['preds'][i]:
        top5s.append(code)
    else:
        top5s.append(0)
    if code == df_result['preds'][i][0]:
        top1s.append(code)
    else:
        top1s.append(0)

100%|██████████| 10966/10966 [00:00<00:00, 54916.40it/s]


In [40]:
pd.concat([df_result, pd.Series(top5s)], axis=1)

,inputs,target,preds,0
0,인슐린신호전달단백질,LA01,"[LA01, LC01, LA06, LC03, LA04]",LA01
1,몰입형 스마트워크,EE04,"[EE10, EE02, EE08, EE14, EE04]",EE04
2,지역단위 교통제어,EI05,"[EI05, EI01, EE12, EE09, EE08]",EI05
3,포도당 운반 단백 단위,LC01,"[LC01, LA07, LC02, LA01, LA02]",LC01
4,사이토카인 아이엘32,LA04,"[LA04, LC01, LC03, LA01, LB07]",LA04
...,...,...,...,...
10961,방사선 반응 추적,EG07,"[EG07, NB01, EA01, LC04, EG04]",EG07
10962,상향식 감축모형,EH10,"[EF01, EF06, EH02, EI09, EI12]",0
10963,멀티레벨 DCDC 컨버터,ED03,"[ED03, EA09, EF05, ED09, EF04]",ED03
10964,항가시아메바성 약제,LC01,"[LC01, LC02, LC03, LA01, LA02]",LC01


In [46]:
a = pd.DataFrame({"inputs" : df_result['inputs'], "target" : df_result["target"], "preds" : df_result["preds"], "correct" : top5s, "top1" : top1s})



In [48]:
a[a['correct'] != 0].drop(['top1'], axis=1) # 0.818

,inputs,target,preds,correct
0,인슐린신호전달단백질,LA01,"[LA01, LC01, LA06, LC03, LA04]",LA01
1,몰입형 스마트워크,EE04,"[EE10, EE02, EE08, EE14, EE04]",EE04
2,지역단위 교통제어,EI05,"[EI05, EI01, EE12, EE09, EE08]",EI05
3,포도당 운반 단백 단위,LC01,"[LC01, LA07, LC02, LA01, LA02]",LC01
4,사이토카인 아이엘32,LA04,"[LA04, LC01, LC03, LA01, LB07]",LA04
...,...,...,...,...
10960,히스톤 H2AX 인산화,LA01,"[LA01, LC01, LA06, LC03, LC02]",LA01
10961,방사선 반응 추적,EG07,"[EG07, NB01, EA01, LC04, EG04]",EG07
10963,멀티레벨 DCDC 컨버터,ED03,"[ED03, EA09, EF05, ED09, EF04]",ED03
10964,항가시아메바성 약제,LC01,"[LC01, LC02, LC03, LA01, LA02]",LC01


In [49]:
a[a['top1'] != 0] # 0.573

,inputs,target,preds,correct,top1
0,인슐린신호전달단백질,LA01,"[LA01, LC01, LA06, LC03, LA04]",LA01,LA01
2,지역단위 교통제어,EI05,"[EI05, EI01, EE12, EE09, EE08]",EI05,EI05
3,포도당 운반 단백 단위,LC01,"[LC01, LA07, LC02, LA01, LA02]",LC01,LC01
4,사이토카인 아이엘32,LA04,"[LA04, LC01, LC03, LA01, LB07]",LA04,LA04
6,형태변형 광학,NB05,"[NB05, NB02, NB06, EA01, NC01]",NB05,NB05
...,...,...,...,...,...
10960,히스톤 H2AX 인산화,LA01,"[LA01, LC01, LA06, LC03, LC02]",LA01,LA01
10961,방사선 반응 추적,EG07,"[EG07, NB01, EA01, LC04, EG04]",EG07,EG07
10963,멀티레벨 DCDC 컨버터,ED03,"[ED03, EA09, EF05, ED09, EF04]",ED03,ED03
10964,항가시아메바성 약제,LC01,"[LC01, LC02, LC03, LA01, LA02]",LC01,LC01
